# Exercises: Warp Basics

Test your understanding of the concepts from `core_01_basics.ipynb`.

In [2]:
import numpy as np
import warp as wp

wp.config.quiet = True
wp.init()

## Exercise 1: Strict Typing

Warp enforces strict typing with no implicit promotion.

**Predict** which of these will succeed and which will raise an error, then run to verify:
1. `wp.float32(1.5) + wp.float32(2.5)`
2. `wp.int8(10) + wp.int32(20)`
3. `wp.float32(1.0) + 2.0`

Fix the failing expression(s) so they all succeed.

In [4]:
wp.float32(1.5) + wp.float32(2.5)
wp.int8(10) + wp.int8(20)
wp.float32(1.0) + wp.float32(2.0)

3.0

## Exercise 2: Custom Matrix Type

Create a 3x2 matrix type of `float64` values using `wp.types.matrix()`. Instantiate it with values 1 through 6 and print it.

Then access the element at row 2, column 1 (0-indexed) and verify it equals `5.0`.

In [22]:
mat32 = wp.types.matrix(shape=(3,2), dtype=wp.float64)
m = mat32(1, 2, 3, 4, 5, 6)
m[2][0]

5.0

## Exercise 3: Array dtype Interpretation

Given the same nested data `((1, 2, 3), (4, 5, 6))`, create:
1. A **2D scalar array** of floats (shape should be `(2, 3)`)
2. A **1D array of vec3** (shape should be `(2,)`)

Print the `shape`, `ndim`, and `dtype` of each to confirm they differ.

In [36]:
print("2D array:")
arr2d = wp.array(shape=(2,3), data=[1,2, 3, 4,5,6])
print(arr2d.shape, arr2d.ndim, arr2d.dtype)

print("1D array:")
arr1d = wp.array(shape=(2,), data=[(1, 2, 3), (4, 5, 6)], dtype=wp.vec3)
print(arr1d.shape, arr1d.ndim, arr1d.dtype)


2D array:
(2, 3) 2 <class 'warp._src.types.int64'>
1D array:
(2,) 1 <class 'warp._src.types.vec3f'>


## Exercise 4: Array Creation Methods

Without looking back at the tutorial, create the following arrays from memory:
1. A 1D array of 5 zeros (dtype `float`)
2. A 1D array of 4 ones (dtype `int`)
3. A 1D array of 6 elements all set to `42.0`
4. A 2D array of shape `(3, 3)` initialized from a NumPy identity matrix

Print each array to verify.

In [43]:
arr1 = wp.zeros(shape=(5,), dtype=wp.float16)
arr2 = wp.ones(shape=(4,), dtype=wp.int16)
arr3 = wp.array(shape=(6,), dtype=wp.float16)
arr3.fill_(42.0)
nparr = np.identity(3)
arr4 = wp.from_numpy(nparr)
print(arr4)


[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## Exercise 5: Structs

Define a `@wp.struct` called `Particle` with three fields:
- `pos`: `wp.vec3`
- `vel`: `wp.vec3`
- `mass`: `float`

Create two particles and store them in a `wp.array`. Print the array.

In [54]:
@wp.struct
class Particle:
    pos: wp.vec3
    vel: wp.vec3
    mass: wp.float16

p1 = Particle()
p1.pos = wp.vec3(1.0, 2.0, 3.0)
p1.vel = wp.vec3(4, 5, 6)
p1.mass = 100

p2 = p1
p1.mass *= 3

arr = wp.array([p1, p2], dtype=Particle)

## Exercise 6: Write a Kernel

Write a kernel `square_kernel` that takes an input array of floats and writes the square of each element to an output array.

Launch it on an array of `[1.0, 2.0, 3.0, 4.0, 5.0]` and verify the output is `[1.0, 4.0, 9.0, 16.0, 25.0]`.

In [68]:
@wp.kernel
def square_kernel(a: wp.array(dtype=float), out: wp.array(dtype=float)):
    i = wp.tid()
    out[i] = a[i] * a[i]

a = wp.array(data=[1.0, 2.0, 3.0, 4.0, 5.0], shape=(5,), dtype=float)
b = wp.empty_like(a)
wp.launch(square_kernel, dim=1, inputs=(a,), outputs=(b,))

## Exercise 7: 2D Kernel Launch

Write a kernel that takes a 2D array of floats (shape `(4, 4)`) and sets each element to `row * 10 + col` using a 2D `wp.tid()`.

Hint: `wp.tid()` returns multiple values for multidimensional launches.

In [77]:
@wp.kernel
def square_kernel_2d(a: wp.array(shape=(4, 4), dtype=float)):
    (r, c) = wp.tid()
    a[r, c] = float(r) * 10.0 + float(c)

data = wp.from_numpy(np.arange(1, 17), dtype=float)
a = wp.array(shape=(4, 4), dtype=float, data=data)
wp.launch(square_kernel_2d, dim=a.shape, outputs=(a,))

## Exercise 8: Device Transfers

1. Create an array `[10, 20, 30]` on CPU
2. Clone it to a CUDA device
3. Verify the GPU clone has the correct values by converting it back to NumPy with `.numpy()`

Hint: `.numpy()` implicitly synchronizes, so no explicit sync call is needed.

In [80]:
arr_cpu = wp.array([10, 20, 30], device="cpu")
arr_cuda = wp.clone(arr_cpu, device="cuda")
arr_np = arr_cuda.numpy()
print(arr_np)

[10 20 30]


## Exercise 9: User Functions

Write a `@wp.func` called `clamp` that takes three float arguments (`val`, `lo`, `hi`) and returns `val` clamped to the range `[lo, hi]`.

Then write a kernel that uses `clamp` to restrict every element of an input array to the range `[-1.0, 1.0]`.

Test with input `[-3.0, -0.5, 0.7, 2.5]` and verify output is `[-1.0, -0.5, 0.7, 1.0]`.

In [86]:
@wp.func
def clamp(val: float, lo: float, hi: float):
    if (val) < lo: return lo
    else: return hi

@wp.kernel
def clamp_kernel(out: wp.array(dtype=float)):
    i = wp.tid()
    out[i] = clamp(out[i], -1.0, 1.0)

arr = wp.array([10.0, -20.0, 30.0], dtype=float)
wp.launch(clamp_kernel, dim=arr.shape, outputs=(arr,))

## Exercise 10: Putting It Together

Write a kernel that normalizes a 1D array of `wp.vec3` vectors (makes each unit length).

Use a `@wp.func` helper that returns `vec / wp.length(vec)` (handle the zero-length case by returning a zero vector).

Test with vectors `(3,0,0)`, `(0,4,0)`, `(1,1,1)`, and `(0,0,0)`.

In [105]:
@wp.func
def normalize_vec(vec: wp.vec3):
    if (wp.length(vec) == 0): return wp.vec3(0.0, 0.0, 0.0)
    else: return vec / wp.length(vec)

@wp.kernel
def normalize_array_kernel(out: wp.array(dtype=wp.vec3)):
    i = wp.tid()
    out[i] = normalize_vec(out[i])


v1 = wp.vec3(3,0,0)
v2 = wp.vec3(0,4,0)
v3 = wp.vec3(1,1,1)
v4 = wp.vec3(0,0,0)
arr = wp.array((v1, v2, v3, v4), dtype=wp.vec3)
wp.launch(normalize_array_kernel, dim=arr.shape, outputs=(arr,))

for v in wp.array.numpy(arr):
    print(v)


[1. 0. 0.]
[0. 1. 0.]
[0.57735026 0.57735026 0.57735026]
[0. 0. 0.]
